In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# Parameters

S_0 = 50           # initial stock price
K = 50             # strike price
T = 0.4167         # time interval in years
N = 30             # number of time steps
sigma = 0.4        # volatility
r = 0.1            # risk-free interest rate

Delta_t = T/N
u = np.exp(sigma * math.sqrt(Delta_t))  # upward movement
d = 1 / u                               # downward movement
p = (np.exp(r * Delta_t) - d) / (u - d) # probability of up move

In [3]:
# Binomial tree of stock prices

def stock_tree(S_0, T, sigma, N):
    gross_up_mov = np.resize(np.arange(N+1), (N+1,N+1))
    gross_down_mov = gross_up_mov.transpose() 
    S = S_0 * np.exp(sigma * math.sqrt(Delta_t) * (gross_up_mov - gross_down_mov))  # only the left upper triangluar part is used actually
    
    return S 


In [4]:
# Binomial tree of American call option prices

def BT_call_price(S_0, K, T, sigma, r, N):
    option_tree = np.zeros((N+1, N+1))
    for i in range(N+1):
        option_tree[i,N-i] = np.maximum(- K + stock_tree(S_0, T, sigma, N)[i,N-i], 0)
    for j in range(N-1, -1, -1):
        for i in range(N-j-1, -1, -1):
            risk_neu_val = np.exp(-r * Delta_t) * (p * option_tree[i, j+1] + (1-p) * option_tree[i+1, j])
            option_tree[i,j] = max(risk_neu_val, - K + stock_tree(S_0, T, sigma, N)[i,j]) # possible early exercise
    
    return option_tree[0,0]
            
BT_call_price(S_0, K, T, sigma, r, N) 

6.074523567932132

In [5]:
# Binomial tree of American put option prices

def BT_put_price(S_0, K, T, sigma, r, N):
    option_tree = np.zeros((N+1, N+1))
    for i in range(N+1):
        option_tree[i,N-i] = np.maximum(K - stock_tree(S_0, T, sigma, N)[i,N-i], 0)
    for j in range(N-1, -1, -1):
        for i in range(N-j-1, -1, -1):
            risk_neu_val = np.exp(-r * Delta_t) * (p * option_tree[i, j+1] + (1-p) * option_tree[i+1, j])
            option_tree[i,j] = max(risk_neu_val, K - stock_tree(S_0, T, sigma, N)[i,j]) # possible early exercise
    
    return option_tree[0,0]
            
BT_put_price(S_0, K, T, sigma, r, N) 

4.263565258847102